In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms

import training_utils as utils
import distillation_methods_module

import numpy as np

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

cuda:0


In [2]:
# Download the training and test sets
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)

train_iter = data.DataLoader(mnist_train, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
test_iter = data.DataLoader(mnist_test, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

100.0%


Extracting ../data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



100.0%


Extracting ../data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



100.0%


Extracting ../data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



100.0%

Extracting ../data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



In [3]:
class Teacher_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 1200)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(1200, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [4]:
class Student_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 600)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(600, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [5]:
teacher = Teacher_Net().to(device)
num_epochs = 10
loss_fn = nn.CrossEntropyLoss(reduction='none').to(device)
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001)
history = utils.train(teacher, train_iter, test_iter, loss_fn, num_epochs, optimizer)

Epoch:		  0
train_metrics:	  (0.6287691440582276, 0.76585)
test_accuracy:	  0.8405
Epoch:		  1
train_metrics:	  (0.4433426361083984, 0.8390833333333333)
test_accuracy:	  0.8498
Epoch:		  2
train_metrics:	  (0.4103693904876709, 0.85055)
test_accuracy:	  0.8612
Epoch:		  3
train_metrics:	  (0.3928386678695679, 0.8566833333333334)
test_accuracy:	  0.8645
Epoch:		  4
train_metrics:	  (0.37783473879496254, 0.8613166666666666)
test_accuracy:	  0.8663
Epoch:		  5
train_metrics:	  (0.37211791191101073, 0.8644666666666667)
test_accuracy:	  0.873
Epoch:		  6
train_metrics:	  (0.3635826767603556, 0.8675)
test_accuracy:	  0.8731
Epoch:		  7
train_metrics:	  (0.3559310776392619, 0.87075)
test_accuracy:	  0.8762
Epoch:		  8
train_metrics:	  (0.3451110579808553, 0.8739833333333333)
test_accuracy:	  0.8738
Epoch:		  9
train_metrics:	  (0.33958055356343586, 0.8735666666666667)
test_accuracy:	  0.8833


In [6]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=5e-3)

d_logits = distillation_methods_module.Logits_Distiller(5, teacher=teacher, student=student, optimizer=optimizer)
d_logits.train(train_iter, test_iter, num_epochs)

Epoch:		  0
train_metrics:	  (2.1197416998135546e-05, 0.707)
test_accuracy:	  0.8053
Epoch:		  1
train_metrics:	  (9.058115893276408e-06, 0.7954333333333333)
test_accuracy:	  0.8234
Epoch:		  2
train_metrics:	  (7.938871916849165e-06, 0.80485)
test_accuracy:	  0.8345
Epoch:		  3
train_metrics:	  (7.5720594283969456e-06, 0.8102666666666667)
test_accuracy:	  0.8306
Epoch:		  4
train_metrics:	  (6.8598131959637e-06, 0.8157833333333333)
test_accuracy:	  0.8408
Epoch:		  5
train_metrics:	  (7.592173454274113e-06, 0.8120166666666667)
test_accuracy:	  0.8325
Epoch:		  6
train_metrics:	  (7.211192446993664e-06, 0.8159833333333333)
test_accuracy:	  0.8353
Epoch:		  7
train_metrics:	  (7.681641302770004e-06, 0.8112833333333334)
test_accuracy:	  0.834
Epoch:		  8
train_metrics:	  (7.671888054270918e-06, 0.81035)
test_accuracy:	  0.8291
Epoch:		  9
train_metrics:	  (7.531920372275636e-06, 0.8137)
test_accuracy:	  0.8338


([0.707,
  0.7954333333333333,
  0.80485,
  0.8102666666666667,
  0.8157833333333333,
  0.8120166666666667,
  0.8159833333333333,
  0.8112833333333334,
  0.81035,
  0.8137],
 [2.1197416998135546e-05,
  9.058115893276408e-06,
  7.938871916849165e-06,
  7.5720594283969456e-06,
  6.8598131959637e-06,
  7.592173454274113e-06,
  7.211192446993664e-06,
  7.681641302770004e-06,
  7.671888054270918e-06,
  7.531920372275636e-06],
 [0.8053,
  0.8234,
  0.8345,
  0.8306,
  0.8408,
  0.8325,
  0.8353,
  0.834,
  0.8291,
  0.8338])

In [16]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

d_features = distillation_methods_module.Features_Distiller(hint_layer=teacher.linear_2, guided_layer=student.linear_2, teacher=teacher, student=student, optimizer=optimizer)
d_features.train(train_iter, test_iter, num_epochs)

Epoch:		  0
train_metrics:	  (0.0009428793812791506, 0.00036666666666666667)
test_accuracy:	  0.0001
Epoch:		  1
train_metrics:	  (0.00037632959087689717, 0.00028333333333333335)
test_accuracy:	  0.0
Epoch:		  2
train_metrics:	  (0.00029673720560967923, 0.00035)
test_accuracy:	  0.0003
Epoch:		  3
train_metrics:	  (0.0002591508763531844, 0.00013333333333333334)
test_accuracy:	  0.0004
Epoch:		  4
train_metrics:	  (0.00023740152915318808, 0.00015)
test_accuracy:	  0.0005
Epoch:		  5
train_metrics:	  (0.00022282475506265958, 0.00015)
test_accuracy:	  0.0007
Epoch:		  6
train_metrics:	  (0.00021345531232655048, 0.00011666666666666667)
test_accuracy:	  0.0006
Epoch:		  7
train_metrics:	  (0.0002068438465396563, 0.00015)
test_accuracy:	  0.0003
Epoch:		  8
train_metrics:	  (0.00019951375455905994, 0.00015)
test_accuracy:	  0.0004
Epoch:		  9
train_metrics:	  (0.00019581584818661213, 0.0001)
test_accuracy:	  0.0005
Epoch:		  0
train_metrics:	  (0.8968599988301595, 0.72535)
test_accuracy:	  0

In [13]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

hint_layers = (teacher.linear_2, teacher.linear_3)
guided_layers = (student.linear_2, student.linear_3)

d_relation = distillation_methods_module.Relations_Distiller(hint_layers=hint_layers, guided_layers=guided_layers, teacher=teacher, student=student, optimizer=optimizer)
d_relation.train(train_iter, test_iter, num_epochs)

Epoch:		  0
train_metrics:	  (232.40440016276042, 0.09845)
test_accuracy:	  0.1


KeyboardInterrupt: 